In [111]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [112]:
def gap(row):
    if abs(row['dualBound']) > 999999999 or abs(row['primalBound']) > 999999999 or \
            row['dualBound'] * row['primalBound'] < 0:
        return 1
    else:
        num = abs(row['primalBound'] - row['dualBound'])
        den = max(abs(row['primalBound']), abs(row['dualBound']))
        return num/den

In [113]:
def get_dfs(test_fldr, base_fldr):
    dfs, bdfs = {}, {}
    for file_name in os.listdir(test_fldr):
        if not file_name.endswith(".csv"):
            continue
        stem = file_name.split('.')[0]
        df = pd.read_csv(os.path.join(test_fldr, file_name))
        bdf = pd.read_csv(os.path.join(base_fldr, file_name))
        df['heuristicPrimalBound'] = np.minimum(df['heuristicPrimalBound'], bdf['heuristicPrimalBound'])
        bdf['heuristicPrimalBound'] = np.minimum(df['heuristicPrimalBound'], bdf['heuristicPrimalBound'])
        df['rootGap'] = (df['heuristicPrimalBound'] - df['rootDualBound']).abs() / \
                        np.maximum(df['heuristicPrimalBound'].abs(), df['rootDualBound'].abs())
        df['gap'] = (df['primalBound'] - df['dualBound']).abs() / np.maximum(df['primalBound'].abs(), df['dualBound'].abs())
        bdf['rootGap'] = (bdf['heuristicPrimalBound'] - bdf['rootDualBound']).abs() / \
                          np.maximum(bdf['heuristicPrimalBound'].abs(), bdf['rootDualBound'].abs())
        bdf['gap'] = (bdf['primalBound'] - bdf['dualBound']).abs() / np.maximum(bdf['primalBound'].abs(), bdf['dualBound'].abs())
        pdf = pd.read_csv(os.path.join('nearest_objective_angles', file_name))
        df = pd.concat([df, pdf], axis=1)
        bdf = pd.concat([bdf, pdf], axis=1)
        df['nearest objective angle'] = df['nearest objective angle']*(1 - (df["vpcGenerator"] == "PRLP"))
        bdf['nearest objective angle'] = df['nearest objective angle']
        dfs[stem] = df
        bdfs[stem] = bdf
    return dfs, bdfs

get_dfs('warm_start', 'benchmark')

({'obj_series_1_perturbed':         lpBound  rootDualBoundPreVpc  rootDualBound    dualBound  \
  0   8489.767168          8490.388086    8507.792285  8521.980144   
  1   8483.143526          8483.438010    8501.333337  8516.387835   
  2   8454.254540          8454.308874    8468.024204  8483.944292   
  3   8509.061017          8510.255047    8523.434333  8538.141828   
  4   8455.441869          8456.294712    8470.643472  8485.989523   
  5   8498.516229          8499.173615    8509.620812  8531.507072   
  6   8478.038733          8479.354697    8482.401684  8508.556075   
  7   8416.119113          8417.155975    8417.155975  8446.386004   
  8   8516.048898          8517.516117    8517.516117  8540.558463   
  9   8499.811051          8499.948123    8510.851628  8533.919741   
  10  8454.131074          8454.241829    8454.435260  8485.289925   
  11  8466.615628          8467.133821    8467.133821  8491.055764   
  12  8442.538430          8442.573729    8442.573729  8470.3161

In [114]:
vpc_dfs, baseline_dfs = get_dfs('warm_start', "benchmark")

In [115]:
# for each instance in compare.index, plot baseline_dfs[rootDualBound] on its own plot
matplotlib.rcParams.update({'font.size': 18})
for instance in vpc_dfs:
    fig=plt.figure(figsize=(18, 6))
    ax1 = plt.subplot(2, 5, 1)
    ax2 = plt.subplot(2, 5, 6, sharex = ax1)
    ax3 = plt.subplot(2, 5, 2, sharey = ax1)
    ax4 = plt.subplot(2, 5, 7, sharex = ax3, sharey = ax2)
    ax5 = plt.subplot(2, 5, 3, sharey = ax1)
    ax6 = plt.subplot(2, 5, 8, sharex = ax5, sharey = ax2)
    ax7 = plt.subplot(2, 5, 4, sharey = ax1)
    ax8 = plt.subplot(2, 5, 9, sharex = ax7, sharey = ax2)
    ax9 = plt.subplot(2, 5, 5, sharey = ax1)
    ax10 = plt.subplot(2, 5, 10, sharex = ax9, sharey = ax2)

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 0]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 0]

    ax1.scatter(range(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax1.scatter(range(len(v)), v['rootGap'], label='VPCs via [1]', c='g', alpha=0.5)
    ax1.axes.get_xaxis().set_visible(False)
    ax1.title.set_text("No Change")
    ax1.set_ylabel('Root Node\nOptimality Gap')

    ax2.scatter(range(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax2.scatter(range(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='VPCs via [1]', c='g', alpha=0.5)
    ax2.set_ylabel('Root Node Processing\nTime (Seconds)')
    ax2.set_xticks([x for x in range(len(b)) if not x%2])

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 1]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 1]

    ax3.scatter(range(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax3.scatter(range(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax3.axes.get_xaxis().set_visible(False)
    ax3.axes.get_yaxis().set_visible(False)
    ax3.title.set_text("1 Degree")

    ax4.scatter(range(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax4.scatter(range(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax4.axes.get_yaxis().set_visible(False)
    ax4.set_xticks([x for x in range(len(b)) if not x%2])

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 3]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 3]

    ax5.scatter(range(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax5.scatter(range(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax5.title.set_text("3 Degrees")
    ax5.axes.get_xaxis().set_visible(False)
    ax5.axes.get_yaxis().set_visible(False)

    ax6.scatter(range(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax6.scatter(range(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax6.axes.get_yaxis().set_visible(False)
    ax6.set_xticks([x for x in range(len(b)) if not x%2])

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 5]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 5]

    ax7.scatter(range(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax7.scatter(range(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax7.title.set_text("5 Degrees")
    ax7.axes.get_xaxis().set_visible(False)
    ax7.axes.get_yaxis().set_visible(False)

    ax8.scatter(range(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax8.scatter(range(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax8.axes.get_yaxis().set_visible(False)
    ax8.set_xticks([x for x in range(len(b)) if not x%2])

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 10]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 10]

    ax9.scatter(range(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax9.scatter(range(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax9.title.set_text("10 Degrees")
    ax9.axes.get_xaxis().set_visible(False)
    ax9.axes.get_yaxis().set_visible(False)

    ax10.scatter(range(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax10.scatter(range(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax10.axes.get_yaxis().set_visible(False)
    ax10.set_xticks([x for x in range(len(b)) if not x%2])
    
    handles, labels = ax1.get_legend_handles_labels()
    more_handles, more_labels = ax3.get_legend_handles_labels()
    handles += more_handles[1:]
    labels += more_labels[1:]
    lgd = fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(.755, -.165))

    text = fig.text(.29, 0.02, "Instance Number", transform=fig.transFigure)

    # fig.tight_layout()
    # plt.savefig(f'{instance}_root_stats.png')
    fig.savefig(f'{instance}_root_stats.png', bbox_extra_artists=(lgd,text), bbox_inches='tight', dpi=300)
    plt.close()

In [23]:
vpc_dfs[stem].loc[vpc_dfs[stem]["intended angle"] == 1]

,lpBound,rootDualBoundPreVpc,rootDualBound,dualBound,heuristicPrimalBound,primalBound,heuristicTime,rootDualBoundTime,terminationTime,maxTerminationTime,completionTime,maxCompletionTime,benchmark,vpcGenerator,terms,vpcGenerationTime,rootGap,gap,nearest objective angle,intended angle
13,8502.158333,8502.709095,8512.046316,8535.105266,8564.016237,8553.693850,0.089369,0.116947,386.586295,395.0,397.0,400.0,0,Farkas,256,8.509546,0.006068,0.002173,0.987810,1
19,8494.963235,8495.115438,8503.574516,8528.103696,8561.048864,8546.344338,0.103637,0.125465,386.590928,395.0,397.0,400.0,0,Farkas,256,8.506085,0.006713,0.002134,1.059338,1
25,8479.876769,8480.084497,8486.542649,8511.317556,8538.080352,8527.286796,0.165455,0.194243,386.644335,395.0,397.0,400.0,0,Farkas,256,8.455206,0.006036,0.001873,0.974335,1
31,8518.142663,8518.290710,8528.659172,8553.802376,8576.753400,8566.558834,0.085508,0.105715,386.568756,395.0,397.0,400.0,0,Farkas,256,8.543063,0.005608,0.001489,1.026453,1
37,8487.159397,8487.491896,8497.160340,8517.628983,8550.593586,8536.202859,0.087931,0.125278,386.653237,395.0,396.0,400.0,0,Farkas,256,8.444404,0.006249,0.002176,0.949019,1
43,8487.715789,8488.084990,8498.623252,8521.130061,8554.497124,8537.586683,0.093628,0.122165,386.648458,395.0,396.0,400.0,0,Farkas,256,8.456940,0.006532,0.001928,0.994188,1
49,8484.960982,8485.326423,8497.042392,8518.720255,8563.102820,8538.403259,0.095833,0.122273,386.462735,395.0,397.0,400.0,0,Farkas,256,8.632928,0.007715,0.002305,1.049042,1


In [7]:
def make_stats(directory, vpc=False):
    dfs = {}
    stats = []
    batch_stats = []
    stems = []
    for csv in os.listdir(directory):
        if not csv.endswith(".csv"):
            continue
        stem = csv.split('.')[0]
        stems.append(stem)
        df = pd.read_csv(os.path.join(directory, csv))
        if vpc:
            df['rootDualBoundTime'] += (df['completionTime'] - df['terminationTime'])  # add time spent making VPCs
        df['primalBound'] = pd.to_numeric(df['primalBound'], errors='coerce')
        df['heuristicPrimalBound'] = pd.to_numeric(df['heuristicPrimalBound'], errors='coerce')
        df['reltime'] = df['completionTime'] / df['maxCompletionTime']
        df['gap'] = df.apply(lambda row: gap(row), axis=1)
        df['nofeas'] = df.apply(lambda row: infeasible(row), axis=1)
        df['batch'] = np.array(df.index)//10 + 1
        stats.append(df[['reltime', 'gap', 'nofeas']].mean())
        df_batch = df.groupby('batch').mean()[['reltime', 'gap', 'nofeas']].reset_index()
        df_batch['instance set'] = stem
        batch_stats.append(df_batch)
        dfs[stem] = df
    df = pd.DataFrame(stats)
    df['instance set'] = stems
    df = df[['instance set', 'reltime', 'gap', 'nofeas']]
    df = df.set_index('instance set')
    df.index.name = None

    df_batch = pd.concat(batch_stats)
    df_batch = df_batch[['instance set', 'batch', 'reltime', 'gap', 'nofeas']]
    df_batch = df_batch.set_index(['instance set', 'batch'])
    df_batch.index.name = None

    return df, dfs, df_batch

In [5]:
def combine(bdf, vdf):
    # join the two dataframes on their index and combine the columns into a multiindex
    return pd.concat([bdf, vdf], axis=1, keys=["without VPCs", "with VPCs"]).swaplevel(0, 1, axis=1).sort_index(axis=1)

In [8]:
baseline_df, baseline_dfs, baseline_batch = make_stats('benchmark_obj_series_1_perturbed', vpc=False)
vpc_df, vpc_dfs, vpc_batch = make_stats('warm_start_obj_series_1_perturbed', vpc=True)
compare = combine(baseline_df, vpc_df)
compare_batch = combine(baseline_batch, vpc_batch)

In [9]:
compare.to_csv('compare.csv', float_format='%.3f')
compare_batch.to_csv('compare_batch.csv', float_format='%.3f')

In [10]:
compare

gap                 nofeas               \
                       with VPCs without VPCs with VPCs without VPCs   
obj_series_1_perturbed   0.00263     0.002546       0.0          0.0   

                         reltime               
                       with VPCs without VPCs  
obj_series_1_perturbed   0.99345       1.0311

In [82]:
compare_batch

gap                 nofeas               \
                               with VPCs without VPCs with VPCs without VPCs   
instance set             batch                                                 
bnd_series_1             1      0.475544     0.410803       0.0          0.0   
                         2      0.465385     0.443894       0.0          0.0   
                         3      0.414930     0.425997       0.0          0.0   
                         4      0.480178     0.408387       0.0          0.0   
                         5           NaN     0.402894       NaN          0.0   
bnd_series_2             1      1.000000     1.000000       1.0          1.0   
                         2      1.000000     1.000000       1.0          1.0   
                         3      1.000000     1.000000       1.0          1.0   
                         4      1.000000     1.000000       1.0          1.0   
                         5      1.000000     1.000000       1.0          1.0   
mat_rhs_bnd_obj_series_1 1      0.400498     0.204845       0.3          0.1   
                         2      0.522210     0.142068       0.4          0.1   
                         3      0.721655     0.156478       0.6          0.1   
                         4      0.191580     0.112554       0.0          0.0   
                         5      0.411795     0.192336       0.3          0.1   
matrix_series_1          1      0.786244     0.785509       0.0          0.0   
                         2      0.780871     0.787701       0.0          0.0   
                         3      0.776623     0.786203       0.0          0.0   
                         4      0.782184     0.786586       0.0          0.0   
                         5      0.778492     0.785980       0.0          0.0   
obj_series_1             1      0.006999     0.005600       0.0          0.0   
                         2      0.010419     0.006617       0.0          0.0   
                         3      0.009581     0.006442       0.0          0.0   
                         4      0.010555     0.006598       0.0          0.0   
                         5      0.010624     0.006417       0.0          0.0   
obj_series_2             1      1.000000     0.853379       0.9          0.8   
                         2      1.000000     1.000000       0.7          0.9   
                         3      0.916446     1.000000       0.9          0.8   
                         4           NaN     1.000000       NaN          0.7   
                         5           NaN     1.000000       NaN          0.7   
rhs_obj_series_1         1      1.000000     0.206138       1.0          0.2   
                         2           NaN     0.358978       NaN          0.3   
                         3           NaN     0.115510       NaN          0.1   
                         4           NaN     0.111426       NaN          0.1   
                         5           NaN     0.007813       NaN          0.0   
rhs_series_1             1      0.333122     0.048140       0.0          0.0   
                         2      1.000000     0.057582       1.0          0.0   
                         3      1.000000     0.047962       1.0          0.0   
                         4      1.000000     0.071238       1.0          0.0   
                         5      1.000000     0.044947       1.0          0.0   
rhs_series_2             1      0.005232     0.005232       0.0          0.0   
                         2      0.005042     0.005042       0.0          0.0   
                         3      0.004999     0.004999       0.0          0.0   
                         4      0.005039     0.005039       0.0          0.0   
                         5      0.005193     0.005193       0.0          0.0   

                                 reltime               
                               with VPCs without VPCs  
instance set             batch                         
bnd_series_1             1  

In [85]:
# for each instance in compare.index, plot baseline_dfs[rootDualBound] on its own plot
for instance in compare.index:
    fig=plt.figure()
    ax1 = plt.subplot(211)
    ax2 = plt.subplot(212, sharex = ax1)

    ax1.plot(baseline_dfs[instance]['dualBound'], label='without VPCs')
    ax1.plot(vpc_dfs[instance]['dualBound'], label='with VPCs')
    ax1.axvline(x=10, color='black', label='begin parametric VPCs')
    # ax1.legend(loc='upper right')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax1.title.set_text(f'{instance}\nDual Bound Comparison with and without VPCs')
    ax1.set_ylabel('Dual Bound')

    ax2.plot(baseline_dfs[instance]['rootDualBoundTime'], label='without VPCs')
    ax2.plot(vpc_dfs[instance]['rootDualBoundTime'], label='with VPCs')
    ax2.axvline(x=10, color='black', label='begin parametric VPCs')
    # ax2.legend(loc='upper right')
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax2.title.set_text(f'{instance}\nRoot Cut Generation Time with and without VPCs')
    ax2.set_xlabel('Instance Number')
    ax2.set_ylabel('Seconds')

    fig.tight_layout()
    plt.savefig(f'{instance}_dual_bounds.png')
    plt.close()